# Two-dimensional Aquifer Thermal Energy Storage (ATES) in a vertical cross-section using the GroundWater Engergy (GWE) package

In this notebook, we will learn how to:
1. Simulate linear ATES in a multi-layer system.
2. Visualize results in a vertical cross-section.
3. Assess the effect of aquitards above and below the aquifer where warm water is injected.

In [ ]:
# import the necessary packages
import numpy as np # the numpy package
import matplotlib.pyplot as plt # the plotting part of matplotlib
plt.rcParams['figure.figsize'] = (5, 3) # set default figure size
import flopy as fp  # import flopy and call it fp
from ipywidgets import interact, FloatSlider, Layout # import some widget functions
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # don't show DeprecationWarning-s in this notebook

## Description of the flow problem
Consider two-dimensional flow in a semi-infinite confined aquifer. For modeling purposes, the aquifer extends from $x=0$ to $x=L$ in the $x$ direction, where $L$ is chosen far enough away not to effect the solution. The aquifer consists of a sand layer covered on top and bottom by a clay layer. The sand layer is modeled with 10 model layers and each clay layer is modeled with 5 model layers. 
Water is injected in the sand layer at the left side at a rate $U$, equally divided over the model layers. 
The head is fixed on the right side of the model to $h_R$ in all layers. All other model boundaries are impermeable to flow of both water and energy. Flow is considered to be at steady state instantaneously. 
The initial temperature is $T_\text{init}$ everywhere. Injection of warmer water with temperature $T_\text{inj}$ starts at $t=0$ and lasts for $t_\text{in}$ days. Buoyancy and viscosity changes are not considered as the temperature differences are small.

### Required changes
The parameter block of the previous notebook needs the following changes:
* Specification of the number of model layers of the clay and sand layers.
* Specification of the hydraulic conductivity as an array with a value for each layer.
 
The `atesmodel` function we created in the previous notebook only needs two modifications:
* Wells inject one tenth of the water in each of the 10 sand layers of the first column.
* Constant head cells are defined in all 20 model layers in the last column of the model.

### Parameters

In [ ]:
# domain size and boundary conditions
L = 150 # length of domain, m
hR = 0 # head at right side of domain

# aquifer parameters
ksand = 35 # hydraulic conductivity sand, m/d
kclay = 0.01 # hydraulic conductivity clay, m/d
Hsand = 20 # aquifer thickness, m
Hclay = 10
theta = 0.3 # porosity, -

# flow parameters
U = 6 # total inflow, m^3/m/d

# energy parameters
rhow = 1000 # density of water, kg/m^3
rhos = 2640 # density of solids, kg/m^3
kappaw = 0.58 * 86400 # thermal conductivity water, J/(d m C)
kappas = 3 * 86400 # thermal conductivity solids, J/(d m C)
cpw = 4180 # specific heat capacity water, J/(kg C)
cps = 710 # specific heat capacity solids, J/(kg C)

# transport
alphaL = 0.1 # longitudinal dispersivity in horizontal direction, m
alphaT = alphaL / 10 # transverse dispersivity is 10 times smaller than longitudinal, m

# temperature
Tinit = 12 # initial temperature, C
Tinj = 17 # temperature injected water, C

# space discretization
delr = 1 # length of cell along row (in x-direction), m
delc = 1 # width of cells normal to plane of flow (in y-direction), m
delz = 2
nlaysand = 10 # number of layers in sand
nlayclay = 5 # number of layers in each clay layer
nlay = 2 * nlayclay + nlaysand
nrow = 1 # number of rows
ncol = round(L / delr) # number of columns, integer\
z = np.linspace(0, -2 * Hclay - Hsand, nlay + 1)
k = np.hstack((kclay * np.ones(nlayclay), ksand * np.ones(nlaysand), kclay * np.ones(nlayclay)))

# time and time discretization
tin = 180 # injection time, d
delt = 1 # time step, d
nstepin = round(tin / delt) # computed number of steps during injection, integer
tout = 180 # extraction time, d
delt = 1 # time step, d
nstepout = round(tout / delt) # computed number of steps during extraction, integer

# model name and workspace
modelname = 'gwe1dml' # name of model
gwfname = modelname + 'f' # name of flow model
gwename = modelname + 'e' # name of energy model
modelws = './' + modelname # model workspace to be used (where MODFLOW will store all the files)

In [ ]:
def atesmodel(ncycle=1):
    # simulation
    sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                              version='mf6', # version of MODFLOW
                              exe_name='mf6', # path to MODFLOW executable
                              sim_ws=modelws, # path to workspace where all files are stored
                             )
    
    # time discretization
    tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                              time_units="DAYS", 
                              nper=2 * ncycle, # number of stress periods 
                              perioddata=ncycle * [[tin, nstepin, 1], # period length, number of steps, timestep multiplier
                                        [tout, nstepout, 1]],
                             )

    # groundwater flow model
    gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                            modelname=gwfname, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                                 filename=gwf.name + '.ims', # file name to store ims
                                 linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                                )                                                                                                
    # register solver
    sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                             model_list=[gwf.name], # list with name of groundwater flow model
                            )   
    
    # discretization
    gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # aquifer properties
    gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                    k=k, # horizontal k value
                                    save_flows=True, # save the flow for all cells
                                   )
        
    # initial condition
    gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                                 strt=hR, # initial head used for iterative solution
                                )
    
    # wells
    wellin = []
    wellout = []
    for ilay in range(nlaysand):
        wellin.append([(nlayclay + ilay, 0, 0), U / nlaysand, Tinj])
        wellout.append([(nlayclay + ilay, 0, 0), -U / nlaysand, Tinj])
    wel_spd = {}
    for i in range(ncycle):
        wel_spd[2 * i] = wellin
        wel_spd[2 * i + 1] = wellout
    gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                                   stress_period_data=wel_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='WEL1', # package name
                                  )
    
    # constant head 
    chd0 = []
    for ilay in range(2 * nlayclay + nlaysand):
        chd0.append([(ilay, 0, ncol - 1), hR, Tinit])
    chd_spd  = {0: chd0} # stress period data
    gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                                   stress_period_data=chd_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='CHD1', # package name
                                  )
        
    # output control
    oc = fp.mf6.ModflowGwfoc(model=gwf, 
                             saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                             head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                            )
    
    # groundwater energy model
    gwe = fp.mf6.ModflowGwe(simulation=sim, # add to simulation called sim
                            modelname=gwename, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwe_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation
                                 filename=gwe.name + '.ims', # must be different than file name of gwf model ims
                                 linear_acceleration="BICGSTAB",
                                ) 
    sim.register_ims_package(solution_file=gwe_ims, 
                             model_list=[gwe.name],
                            )
    
    # discretization
    gwe_dis = fp.mf6.ModflowGwedis(model=gwe, # add to gwe model
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # initial condition
    gwe_ic = fp.mf6.ModflowGweic(model=gwe, 
                                 strt=Tinit, # initial temperature
                                ) 
    
    # advection
    adv = fp.mf6.ModflowGweadv(model=gwe,  
                               #scheme="upstream", # use the upstream method
                               scheme="tvd", # use the upstream method
                               pname='ADV1',
                              )
    
    # energy storage package
    gwe_sto = fp.mf6.ModflowGweest(model=gwe, 
                                   porosity=theta, # porosity
                                   heat_capacity_water=cpw,
                                   density_water=rhow,
                                   heat_capacity_solid=cps,
                                   density_solid=rhos,
                                   pname="EST",
                                   save_flows=True,
                                  )
    
    # conduction
    dsp = fp.mf6.ModflowGwecnd(model=gwe, 
                               xt3d_off=True,
                               alh=alphaL, # longitudinal dispersivity
                               ath1=alphaT, # transverse dispersivity
                               ktw=kappaw,
                               kts=kappas,
                               pname='CND', 
                              )
    
    # source sink mixing
    sourcelist = [("WEL1", "AUX", "TEMPERATURE"), ("CHD1", "AUX", "TEMPERATURE")] # list of (pname, 'AUX', 'TEMPERATURE')
    ssm = fp.mf6.ModflowGwessm(model=gwe, 
                               sources=sourcelist, 
                               save_flows=True,
                               pname='SSM1', 
                              )
    
    # output control
    oc = fp.mf6.ModflowGweoc(model=gwe,
                             saverecord=[("TEMPERATURE", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwename}.cbc", # file name where all budget output is stored
                             temperature_filerecord=f"{gwename}.ucn", # file name where all concentration output is stored
                            )
    
    # interaction between gwf and gwe
    fp.mf6.ModflowGwfgwe(simulation=sim, 
                     exgtype="GWF6-GWE6", 
                     exgmnamea=gwf.name, # name of groundwater flow model 
                     exgmnameb=gwe.name, # name of transport model
                     filename=f"{modelname}.gwfgwe",
                    );
    
    # write and solve model
    sim.write_simulation(silent=True)
    success, _ = sim.run_simulation(silent=True) 
    if success == 1:
        print('Model solved successfully')
    else:
        print('Solve failed')

    # read concentration output
    tempobj = gwe.output.temperature() # get handle to binary concentration file
    temp = tempobj.get_alldata().squeeze() # get the concentration data from the file
    times = np.array(tempobj.get_times()) # get the times and convert to array

    return temp, times, gwf, gwe

In [ ]:
# run model
ncycle = 3
temp, times, gwf, gwe = atesmodel(ncycle)

Create a cross-sectional figure using `flopy` commands and add a slider to change the time.

In [ ]:
def plot(t):
    tindex = int(t / delt) - 1
    plt.figure(figsize=(10, 4))
    ax = plt.subplot(111, xlabel='x (m)', ylabel='z (m)')
    pxs = fp.plot.PlotCrossSection(model=gwe, line={"row": 0}, ax=ax) # contour plot for gwe, the transport model
    pa = pxs.plot_array(temp[tindex], vmin=12, vmax=17, cmap='Spectral_r') # plot an array by colored grid blocks
    cb = plt.colorbar(pa)
    cb.set_label("temperature (\u2103)")
    plt.axhline(-10, color='w', linestyle=':')
    plt.axhline(-30, color='w', linestyle=':')

interact(plot, t=FloatSlider(value=delt, min=delt, max=ncycle * (tin + tout), 
                                description='time (days):', readout_format='.1f',
                                step=delt, layout=Layout(width='80%')));

## Thermal recovery efficiency
The thermal recovery of each cycle is computed below

In [ ]:
energy_injected = U * (Tinj - Tinit) * cpw * rhow * tin

for i in range(ncycle):
    istart = nstepin + i * (nstepin + nstepout)
    iend = istart + nstepout
    energy_extracted = np.sum(U / nlaysand * 
                              (temp[istart: iend, nlayclay: nlayclay + nlaysand, 0] - Tinit)
                               * cpw * rhow * delt)
    
    recovery_efficiency = energy_extracted / energy_injected
    print(f'cycle, recovery efficiency, i, {recovery_efficiency * 100:.2f}%')

This model is a better approximation of a real system than the one-dimensional flow problem of the previous notebook. As we saw, a significant amount of energy is lost through conduction into the overlying and underlying clay layers. Bear in mind, this model is still approximate. Some of the approximations are:
* The top of the upper clay layer and the bottom of the lower clay layer are impermeable for energy (and water) while some energy will likely be lost through those boundaries.
* The solution will likely change some when smaller cell sizes and time steps are used.
* The flow into the sand layer is equally distributed over the thickness (although this is probably quite reasonable, as the hydraulic conductivity of the clay layer is very small compared to the sand layer).
* The thermal parameters of the solids of the sand and clay are taken equal here for simplicity.
* Some energy may be lost across the right model boundary. A longer model may reduce that loss.